# AI Home Staging - Retrieval Demo

This notebook demonstrates the image retrieval system for AI home staging, where we find similar staged bedroom designs based on an empty bedroom image.

In [1]:
# Fix the module import issue by adding parent directory to Python path
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
import torch
from src.data_loader import load_image
from src.feature_extractor import FeatureExtractor
from src.retrieval import retrieve

print("✅ Successfully imported all modules!")
print(f"📁 Current working directory: {os.getcwd()}")
print(f"🐍 Python path includes: {sys.path[-1]}")

✅ Successfully imported all modules!
📁 Current working directory: C:\Users\haari\STAGING WORK\ai-home-staging\Notebooks
🐍 Python path includes: C:\Users\haari\STAGING WORK\ai-home-staging


In [2]:
# Check if model files exist
models_dir = "../models"
staged_embs_path = os.path.join(models_dir, "staged_embs.npy")
staged_names_path = os.path.join(models_dir, "staged_names.npy")

print(f"🔍 Checking for model files...")
print(f"   staged_embs.npy exists: {os.path.exists(staged_embs_path)}")
print(f"   staged_names.npy exists: {os.path.exists(staged_names_path)}")

# Load models & data if they exist
if os.path.exists(staged_embs_path) and os.path.exists(staged_names_path):
    staged_embs = np.load(staged_embs_path)
    staged_names = np.load(staged_names_path)
    print(f"✅ Loaded {len(staged_embs)} staged embeddings")
    print(f"✅ Loaded {len(staged_names)} staged image names")
else:
    print("⚠️  Model files not found. You may need to run the embedding generation first.")
    staged_embs = None
    staged_names = None

🔍 Checking for model files...
   staged_embs.npy exists: True
   staged_names.npy exists: True
✅ Loaded 2 staged embeddings
✅ Loaded 2 staged image names


In [3]:
# Initialize the feature extractor model
print("🤖 Initializing FeatureExtractor model...")
model = FeatureExtractor()
model.eval()
print("✅ Model initialized successfully!")

# Test the model with a dummy input
with torch.no_grad():
    dummy_input = torch.randn(1, 3, 256, 256)
    dummy_output = model(dummy_input)
    print(f"🧪 Model test output shape: {dummy_output.shape}")

🤖 Initializing FeatureExtractor model...
✅ Model initialized successfully!
🧪 Model test output shape: torch.Size([1, 2048])


In [4]:
# Check if query image exists
query_image_path = "../data/empty bedroom/IMG_0938.jpeg"
print(f"🔍 Checking for query image: {query_image_path}")
print(f"   Image exists: {os.path.exists(query_image_path)}")

if os.path.exists(query_image_path) and staged_embs is not None:
    # Load and process the query image
    print("📸 Loading query image...")
    img = load_image(query_image_path).unsqueeze(0)
    print(f"   Image shape: {img.shape}")
    
    # Extract features from the query image
    print("🔍 Extracting features from query image...")
    with torch.no_grad():
        q_emb = model(img).cpu().numpy()
    print(f"   Query embedding shape: {q_emb.shape}")
    
    # Perform retrieval
    print("🔍 Performing similarity search...")
    idxs, sims = retrieve(q_emb, staged_embs, top_k=3)
    
    print("\n🏆 Top 3 matches:")
    for i, (idx, sim) in enumerate(zip(idxs, sims)):
        print(f"   {i+1}. {staged_names[idx]} (similarity: {sim:.4f})")
else:
    print("⚠️  Cannot perform retrieval - missing query image or model files")

🔍 Checking for query image: ../data/empty bedroom/IMG_0938.jpeg
   Image exists: True
📸 Loading query image...
   Image shape: torch.Size([1, 3, 256, 256])
🔍 Extracting features from query image...
   Query embedding shape: (1, 2048)
🔍 Performing similarity search...

🏆 Top 3 matches:
   1. IMG_4396.jpeg (similarity: 0.8182)
   2. IMG_4398.jpeg (similarity: 0.8075)
